In [6]:
"""###################################################################
Module to be loaded before running the script:
- ANTs/git
- qbatch/git

Note:
- 100 nii.gz label files needs ~8 cores of processors to run

###################################################################"""
print "This box contains notes for myself"

This box contains notes for myself


In [7]:
# ====================================================================
# IMPORT LIBRARIES
# ====================================================================
import os
import subprocess
import sys
import glob #for wildcard matching
import time



In [8]:
# ====================================================================
# Setup directories, paths and filenames
# - All paths must be ABSOLUTE!
# - Refer to 'glob' manual to see the wildcard documentation for file-
#        name matching
# 
# ====================================================================
""" Note on directory hierarchy for the input, reference and transform files 
parent_dir
    |
    +-- subj_dir_1 (as indicated by dir_wc)
    |        |
    |        +- file_of_interest (as indicated by file_wc)
    |
    +-- subj_dir_2 ...
"""

# =====================================
# Input related
# =====================================
######## Input file paths ########
# Path to the parent directory of the input files
in_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/tract2voxel_probability_labels/model_space'
# Wildcard / name of the individual subject directories
in_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
# Wildcard / name of the input file
in_file_wc='[0-9]*[0-9]_region_seg_idx_modelSpace.nii.gz'


# =====================================
# Output file path
# =====================================
######## Output file path ########
output_file_path='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/tract2voxel_probability_labels/model_space/region_seg5_idx_modelSpace_majVote.nii.gz'

######## Job document paths ########
jobList_dir='/data/chamal/projects/anthony/qbatch_jobDocs'
jobList_name='seg5_voxelizedTracts_segIdxMajVote_jobList.sub'


# =====================================
# ImageMath related
# =====================================
DIMENSIONALITY='3'
OPERATION='MajorityVoting'


In [9]:
# ====================================================================
# Save the filenames into lists
# ====================================================================

#### Initilize and save input files
in_file_list=glob.glob(os.path.join(in_parent_dir,in_dir_wc,in_file_wc))
in_file_list.sort()

#Printout
print "File initilization stats:"
print "\tTotal input files: %d" % len(in_file_list)


File initilization stats:
	Total input files: 100


In [10]:
# ====================================================================
# CUSTOMIZATIONS
# ====================================================================
# Submission related
QSUB = True #Submit on cluster?
SUBMIT = True #Automatically send in the jobList or qsub myself?
# QBatch related
WAITTIME='10:00:00'
CHUNKSIZE='1'
PROCESSOR_PER_JOB='8' #Defalut = 1 

# ====================================================================
# Make jobList and optionally submit
# ====================================================================



#Check that output file is not created already
if os.path.isfile(output_file_path):
    print 'Output file already present. Aborting.'
    sys.exit()

##### MAKE THE COMMAND #####
#Make the base command
cmd = ['ImageMath',DIMENSIONALITY,output_file_path,OPERATION]
#Add the input files
cmd += in_file_list

#If this is meant to be submitted on the cluster, generate job document
if QSUB == True:
    #Open output stream
    jobList_path = os.path.join(jobList_dir,jobList_name)
    jobList_outStream = open(jobList_path,'w')
    #Annotate jobList
    jobList_outStream.write("# Job doc generated on: %s\n" % time.strftime("%Y%d%m-%H%M")) 
    jobList_outStream.write("# antsApplyTransforms specs:\n")
    jobList_outStream.write("# \tDimensionality: %s\n" % DIMENSIONALITY)
    jobList_outStream.write("# \tOperation: %s\n"% OPERATION)
    jobList_outStream.write("# Qbatch specs:\n")
    jobList_outStream.write("# \tAutosubmitted: %s\n" % str(SUBMIT))
    jobList_outStream.write("# \tWaittime: %s\n" % WAITTIME)
    jobList_outStream.write("# \tChunksize: %s\n" % CHUNKSIZE)
    jobList_outStream.write("# \tProcessors per job: %s\n\n" % PROCESSOR_PER_JOB)
    #Create command
    jobList_outStream.write(' '.join(cmd)+'\n\n\n')
    #Annotate with additional information about input files
    jobList_outStream.write("# Input files parent folder: %s\n" % in_parent_dir)
    jobList_outStream.write("# Number of input files: %d\n" % len(in_file_list))
    jobList_outStream.write("# Input files:\n")
    for f in in_file_list:
        jobList_outStream.write("#\t%s\n"%f)
    #Close output stream
    jobList_outStream.close()
    
    #Autosubmit if indicated
    if SUBMIT==True:
        sub_cmd = ['qbatch','-w',WAITTIME,'-c',CHUNKSIZE,'--ppj',PROCESSOR_PER_JOB,jobList_path]
        subprocess.call(sub_cmd)
#Else, run locally
else:
    subprocess.call(cmd)
    


